In [6]:
import numpy as np
import math
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import calibrate_power as cbp

from scipy import signal

import sys
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis\read")

from read_scope_data import read_trc_data

%matplotlib widget

In [ ]:
ifn = r"C:\data\epfl\CRDS\scope trc\bdcageAnt\C1-300W-1mT-00000.trc"
Iant, tarr = read_trc_data(ifn)
Iant *= 8 # Pearson 8A/V

ifn = r"C:\data\epfl\CRDS\scope trc\solAnt\C3-500W-36mT-00000.trc"
light, tarr = read_trc_data(ifn)
print(tarr.shape, light.shape)

ifn = r"C:\data\epfl\CRDS\scope trc\solAnt\C4-500W-36mT-00000.trc"
Vtrig, tarr = read_trc_data(ifn)

In [ ]:
fig, ax = plt.subplots(nrows=3, sharex=True)
ax[0].plot(tarr*1e3, Iant)
ax[0].set_ylabel('Iant (A)')

ax[1].plot(tarr*1e3, light)
ax[1].set_ylabel('Light')

ax[2].plot(tarr*1e3, Vtrig)
ax[2].set_ylabel('TTL')
ax[2].set_xlabel('time (ms)')

start = 22000
stop = 24000
ax[1].plot(tarr[start:stop]*1e3, light[start:stop]) # plot range to take average
lavg = numpy.average(light[start:stop]) * 1e3 # take average light signal conver V to mV
print(cbp.ltop_solenoid(lavg, 35)) # Calibrated power according to CW light signal

In [ ]:
ifn = r"C:\data\epfl\CRDS\solenoid\20230601_0.5mTorr_0.007lnmin.xlsx"

# xlsx is only supported by openpyxl special package because they saved the excel on a very old version ugh
excel_data_df = pd.read_excel(ifn, sheet_name=0, engine='openpyxl', header=[0])

hdr_ls = excel_data_df.columns.ravel()
print(hdr_ls)

In [ ]:
tarr_crds = excel_data_df[hdr_ls[0]].to_numpy()
sig_crds = excel_data_df[hdr_ls[1]].to_numpy()

tarr_crds = tarr_crds[~numpy.isnan(sig_crds)]
sig_crds = sig_crds[~numpy.isnan(sig_crds)]

print(len(sig_crds))

In [ ]:
fig = plt.figure()
plt.scatter(tarr_crds, sig_crds)

# signal baseline
start = 200
stop = 240
plt.scatter(tarr_crds[start:stop], sig_crds[start:stop])
sig_base = numpy.average(sig_crds[start:stop])

# signal active-glow equilibrium value
start = -70
stop = -20
plt.scatter(tarr_crds[start:stop], sig_crds[start:stop])
sig_val = numpy.average(sig_crds[start:stop])

print('%.2e' %(sig_val-sig_base))

In [ ]:
plt.close(fig)

In [ ]:
fig, ax = plt.subplots(nrows=3, sharex=True)
fig.suptitle('5mTorr/0.67Pa 580W')

ax[0].plot(tarr*1e3, light)
ax[0].set_ylabel('Light')

ax[1].plot(tarr*1e3, Vtrig)
ax[1].set_ylabel('TTL')

tarr_fake = tarr_crds + tarr_crds[-1]
tarr_crds_mod = numpy.hstack((tarr_crds, tarr_fake)) #+tarr_crds[100]
sig_crds_mod = numpy.tile(sig_crds , 2) #numpy.roll(sig_crds,-100)

ax[2].plot(tarr_crds_mod*1e3, sig_crds_mod)
ax[2].set_ylabel('crds signal')
ax[2].set_xlabel('time (ms)')